# Volatility Trading
The strangle strategy involves simultaneously buying both a call option and a put option on the same underlying asset, with different strike prices but the same expiration date. By employing this strategy, traders aim to profit from significant price swings or increased volatility while managing their risk exposure.

In [91]:
import math
from scipy.stats import norm

class EuropeanCallOption:
    def __init__(self, spot_price, strike_price, risk_free_rate, volatility, time_to_maturity):
        """
        Initializes a European Call Option.

        Parameters:
            spot_price (float): Current spot price of the underlying asset.
            strike_price (float): Strike price of the option.
            risk_free_rate (float): Risk-free interest rate, typically an annualized rate.
            volatility (float): Volatility of the underlying asset's returns.
            time_to_maturity (float): Time to maturity of the option in years.
        """
        self.spot_price = spot_price
        self.strike_price = strike_price
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility
        self.time_to_maturity = time_to_maturity
        
    @property
    def d1(self):
        """
        Calculates the d1 parameter used in the Black-Scholes-Merton model.

        Returns:
            float: The calculated d1 value.
        """
        d1 = (np.log(self.spot_price/self.strike_price) \
                  + (self.risk_free_rate + self.volatility**2/2)*self.time_to_maturity)\
                  / (self.volatility*np.sqrt(self.time_to_maturity))
        return d1
    
    @property
    def d2(self):
        """
        Calculates the d2 parameter used in the Black-Scholes-Merton model.

        Returns:
            float: The calculated d2 value.
        """
        d2 = self.d1 - self.volatility * np.sqrt(self.time_to_maturity)
        return d2

    @property
    def price(self):
        """
        Calculates the price of the European Call Option using the Black-Scholes-Merton model.

        Returns:
            float: The calculated option price.
        """
        option_price = self.spot_price * norm.cdf(self.d1) \
                        - self.strike_price * math.exp(-self.risk_free_rate * self.time_to_maturity) * norm.cdf(self.d2)
        return option_price
    
    @property
    def delta(self):
        original_spot = self.spot_price
        s_plus = self.spot_price*1.001
        s_minus = self.spot_price*0.999
        self.spot_price= s_plus
        theovalue_plus = self.price
        self.spot_price = s_minus
        theovalue_minus = self.price
        self.spot_price = original_spot
        return (theovalue_plus - theovalue_minus)/(s_plus-s_minus)
    
    
class EuropeanPutOption:
    def __init__(self, spot_price, strike_price, risk_free_rate, volatility, time_to_maturity):
        """
        Initializes a European Call Option.

        Parameters:
            spot_price (float): Current spot price of the underlying asset.
            strike_price (float): Strike price of the option.
            risk_free_rate (float): Risk-free interest rate, typically an annualized rate.
            volatility (float): Volatility of the underlying asset's returns.
            time_to_maturity (float): Time to maturity of the option in years.
        """
        self.spot_price = spot_price
        self.strike_price = strike_price
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility
        self.time_to_maturity = time_to_maturity
        
    @property
    def d1(self):
        """
        Calculates the d1 parameter used in the Black-Scholes-Merton model.

        Returns:
            float: The calculated d1 value.
        """
        d1 = (np.log(self.spot_price/self.strike_price) \
                  + (self.risk_free_rate + self.volatility**2/2)*self.time_to_maturity)\
                  / (self.volatility*np.sqrt(self.time_to_maturity))
        return d1
    
    @property
    def d2(self):
        """
        Calculates the d2 parameter used in the Black-Scholes-Merton model.

        Returns:
            float: The calculated d2 value.
        """
        d2 = self.d1 - self.volatility * np.sqrt(self.time_to_maturity)
        return d2

    @property
    def price(self):
        """
        Calculates the price of the European Call Option using the Black-Scholes-Merton model.

        Returns:
            float: The calculated option price.
        """
        option_price = -self.spot_price * norm.cdf(-self.d1) \
                        + self.strike_price * math.exp(-self.risk_free_rate * self.time_to_maturity) * norm.cdf(-self.d2)
        return option_price
    
    @property
    def delta(self):
        original_spot = self.spot_price
        s_plus = self.spot_price*1.001
        s_minus = self.spot_price*0.999
        self.spot_price= s_plus
        theovalue_plus = self.price
        self.spot_price = s_minus
        theovalue_minus = self.price
        self.spot_price = original_spot
        return (theovalue_plus - theovalue_minus)/(s_plus-s_minus)
    
    @property
    def gamma(self):
        return norm.pdf(self.d1)\
                /(self.spot_price*self.volatility*np.sqrt(self.time_to_maturity)) 
    
class Index:
    def __init__(self, price):
        self.price = -price

### Portfolio class

In [106]:
class StraddlePortfolio:
    def __init__(self, holdings):
        self.holdings = holdings
        
    @property
    def value(self):
        return sum(holding.price for holding in self.holdings)
    
    def update_price(self, new_price):
        for holding in self.holdings:
            holding.spot_price = new_price
            
    def update_vol(self, new_vol):
        for holding in self.holdings:
            holding.volatility = new_vol
            
    def update_mat(self, new_mat):
        for holding in self.holdings:
            holding.time_to_maturity = new_mat
    
    @property
    def delta(self):
        return sum(holding.delta for holding in self.holdings)

In [107]:
import yfinance as yf
from datetime import datetime
import numpy as np
import pandas as pd

sp500 = yf.download('^GSPC','2023-01-01',datetime.today(), progress=False)['Close'].rename('S&P 500')
volatility_series = sp500.rolling(20).apply(lambda x: np.std(np.log(1+x)))
first_volatility = volatility_series.dropna()[0]
volatility_series.fillna(first_volatility, inplace=True)
maturity_date_series = (pd.to_datetime('2023-06-15') - sp500.index).days / 250

In [108]:
call_option = EuropeanCallOption(sp500[0], sp500[0], 0.02, volatility_series[0], maturity_date_series[0])
put_option = EuropeanPutOption(sp500[0], sp500[0], 0.02, volatility_series[0], maturity_date_series[0])

straddle = StraddlePortfolio(holdings=[call_option, put_option])